In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp

In [ ]:
temp = cudf.read_parquet('./temp.parquet/*')[['GISJOIN','JLZE002','JLZE026']]

In [3]:
temp

,GISJOIN,JLZE002,JLZE026
0,1000300106002025,0.362652,0.637348
1,1000300106002025,0.362652,0.637348
2,1000300106002025,0.362652,0.637348
3,1000300106002025,0.362652,0.637348
4,1000300106002025,0.362652,0.637348
...,...,...,...
308745304,56004300003013056,0.445183,0.554817
308745305,56004300003013056,0.445183,0.554817
308745306,56004300003013056,0.445183,0.554817
308745307,56004300003013056,0.445183,0.554817


In [4]:
temp1 = temp.groupby(by='GISJOIN').mean()
temp2 = temp.groupby(by='GISJOIN').count()

temp1.JLZE002 = temp1.JLZE002
temp1.JLZE026 = temp1.JLZE026
temp1 = temp1.nans_to_nulls()
temp1 = temp1.fillna(0.5)

In [5]:
del(temp)

In [6]:
temp1['total_count'] = temp2.JLZE002

In [7]:
del(temp2)

In [8]:
temp1['Male'] = (temp1.JLZE002 * temp1.total_count).round().astype('int')

In [9]:
temp1['Female'] = (temp1.total_count - temp1.Male).astype('int')

In [10]:
temp1

,JLZE002,JLZE026,total_count,Male,Female
GISJOIN,,,,,
1000100201001000,0.427788,0.572212,61,26,35
1000100201001003,0.427788,0.572212,75,32,43
1000100201001005,0.427788,0.572212,1,0,1
1000100201001007,0.427788,0.572212,23,10,13
1000100201001009,0.427788,0.572212,1,0,1
...,...,...,...,...,...
56004509513003096,0.536510,0.463490,12,6,6
56004509513003104,0.536510,0.463490,160,86,74
56004509513003112,0.536510,0.463490,3,2,1


In [11]:
(temp1.Male.sum() + temp1.Female.sum()) == temp1.total_count.sum()

True

In [12]:
temp1 = temp1.drop(['JLZE002', 'JLZE026', 'total_count'])

In [13]:
temp1

,Male,Female
GISJOIN,,
1000100201001000,26,35
1000100201001003,32,43
1000100201001005,0,1
1000100201001007,10,13
1000100201001009,0,1
...,...,...
56004509513003096,6,6
56004509513003104,86,74
56004509513003112,2,1


In [14]:
df_males = cudf.DataFrame({
    'GISJOIN': temp1.index.repeat(temp1.Male).as_column()
})
df_males['sex'] = 0
df_females = cudf.DataFrame({
    'GISJOIN': temp1.index.repeat(temp1.Female).as_column()
})
df_females['sex'] = 1

In [15]:
df_final = cudf.concat([df_males, df_females]).reset_index()[['GISJOIN','sex']]
del(df_males)
del(temp1)
del(df_females)

In [16]:
# df_final.index= df_final.GISJOIN
# df_final.drop_column('GISJOIN')
df_final

,GISJOIN,sex
0,1000100201001000,0
1,1000100201001000,0
2,1000100201001000,0
3,1000100201001000,0
4,1000100201001000,0
...,...,...
308745304,56004509513003128,1
308745305,56004509513003128,1
308745306,56004509513003128,1
308745307,56004509513003128,1


In [17]:
df_final.to_parquet('temp2.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "


### sort temp.parquet

In [3]:
df = cudf.read_parquet('./temp.parquet/*')[['GISJOIN', 'x', 'y']]
df = df.sort_values('GISJOIN')
df.to_parquet('sorted_temp.parquet')

In [2]:
df = cudf.read_parquet('./temp2.parquet/*')[['GISJOIN', 'sex']]
df = df.sort_values('GISJOIN')
df.to_parquet('sorted_temp2.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "


### merge sex with lat-long table

In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp

In [2]:
df = cudf.read_parquet('./sorted_temp.parquet/*')

In [3]:
df.head(100)

,GISJOIN,x,y
900512,1000100201001000,-9.627034e+06,3.825003e+06
900513,1000100201001000,-9.626834e+06,3.825034e+06
900514,1000100201001000,-9.626683e+06,3.825079e+06
900515,1000100201001000,-9.627138e+06,3.825075e+06
900516,1000100201001000,-9.627112e+06,3.824731e+06
...,...,...,...
840098,1000100201001003,-9.626974e+06,3.824396e+06
840099,1000100201001003,-9.627868e+06,3.825105e+06
840100,1000100201001003,-9.627994e+06,3.825250e+06
840101,1000100201001003,-9.628304e+06,3.825228e+06


In [4]:
df_with_sex = cudf.read_parquet('./sorted_temp2.parquet/*').reset_index().drop(['index'])

In [5]:
df_with_sex.head(100)

,GISJOIN,sex
0,1000100201001000,0
1,1000100201001000,0
2,1000100201001000,0
3,1000100201001000,0
4,1000100201001000,0
...,...,...
95,1000100201001003,1
96,1000100201001003,1
97,1000100201001003,1
98,1000100201001003,1


In [6]:
df['GISJOIN1'] = df_with_sex.GISJOIN.values

In [8]:
df.query('GISJOIN != GISJOIN1')

,GISJOIN,x,y,GISJOIN1


In [15]:
df['sex'] = df_with_sex.sex.astype('int8')

In [10]:
df.drop_column('GISJOIN1')

In [16]:
df.dtypes

GISJOIN      int64
x          float64
y          float64
sex           int8
dtype: object

In [17]:
df

,GISJOIN,x,y,sex
900512,1000100201001000,-9.627034e+06,3.825003e+06,0
900513,1000100201001000,-9.626834e+06,3.825034e+06,0
900514,1000100201001000,-9.626683e+06,3.825079e+06,0
900515,1000100201001000,-9.627138e+06,3.825075e+06,0
900516,1000100201001000,-9.627112e+06,3.824731e+06,0
...,...,...,...,...
308744617,56004509513003128,-1.159921e+07,5.441573e+06,1
308744618,56004509513003128,-1.159932e+07,5.441618e+06,1
308744619,56004509513003128,-1.159941e+07,5.441687e+06,1
308744620,56004509513003128,-1.159941e+07,5.441688e+06,1


In [18]:
df.to_parquet('all_states_us_lat_long_sex.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "
